# run through existing readouts and design adaptors

In [1]:
%run "E:\Users\puzheng\Documents\Startup_py3.py"
sys.path.append(r"E:\Users\puzheng\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

10364


In [2]:
# biopython imports
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
from Bio.SeqRecord import SeqRecord
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML

# Load readouts

In [3]:
readout_folder = r'\\10.245.74.212\Chromatin_NAS_2\Libraries\Readouts'
ref_files = [_fl for _fl in os.listdir(readout_folder) if 'designed_readouts' in _fl]

ref_readout_dict = {}
ref_readout_record_dict = {}
for _fl in ref_files:
    _channel = int(_fl.split('designed_readouts_')[1].split('.fasta')[0])
    _ref_readout_names = []
    _ref_readout_records = []
    with open(os.path.join(readout_folder, _fl), 'r') as _rd_handle:
        for _readout in SeqIO.parse(_rd_handle, "fasta"):
            _ref_readout_names.append(_readout.id)
            _ref_readout_records.append(_readout)
    ref_readout_dict[_channel] = _ref_readout_names
    ref_readout_record_dict[_channel] = _ref_readout_records

## load readout sites

In [17]:
# load readout sites
adaptor_folder = r'\\10.245.74.212\Chromatin_NAS_2\Libraries\Adaptors'
readout_site_file = os.path.join(adaptor_folder, 'Readout_sites.fasta')
readout_sites = []
readout_site_dict = {}
readout_channels = ['750', '647', '561']

with open(readout_site_file, 'r') as _rd_handle:
    for _i, _readout in enumerate(SeqIO.parse(_rd_handle, "fasta")):
        readout_sites.append(_readout)
        readout_site_dict[readout_channels[_i]] = _readout

print(readout_site_dict)

{'750': SeqRecord(seq=Seq('TTTGCACTGCCGTCCTTGAC', SingleLetterAlphabet()), id='Stv_82', name='Stv_82', description='Stv_82 cy7 rev-com_last20', dbxrefs=[]), '647': SeqRecord(seq=Seq('GATCCGATTGGAACCGTCCC', SingleLetterAlphabet()), id='Stv_1', name='Stv_1', description='Stv_1 cy5 rev-com_last20', dbxrefs=[]), '561': SeqRecord(seq=Seq('TGCGAACTGTCCGGCTTTCA', SingleLetterAlphabet()), id='Stv_79', name='Stv_79', description='Stv_79 cy3 rev-com_last20', dbxrefs=[])}


In [6]:
# generate adaptor
reload(library_tools.readouts)
from ImageAnalysis3.library_tools.readouts import Generate_adaptors

In [19]:
readout_site_dict

{'750': SeqRecord(seq=Seq('TTTGCACTGCCGTCCTTGAC', SingleLetterAlphabet()), id='Stv_82', name='Stv_82', description='Stv_82 cy7 rev-com_last20', dbxrefs=[]),
 '647': SeqRecord(seq=Seq('GATCCGATTGGAACCGTCCC', SingleLetterAlphabet()), id='Stv_1', name='Stv_1', description='Stv_1 cy5 rev-com_last20', dbxrefs=[]),
 '561': SeqRecord(seq=Seq('TGCGAACTGTCCGGCTTTCA', SingleLetterAlphabet()), id='Stv_79', name='Stv_79', description='Stv_79 cy3 rev-com_last20', dbxrefs=[])}

## generate adaptors

In [24]:
adaptor_dict = {}

for _ch, _records in ref_readout_record_dict.items():
    print(_ch)
    _adaptors = Generate_adaptors(_records, [readout_site_dict[str(_ch)]])
    
    adaptor_dict[_ch] = _adaptors


561
647
750


In [28]:
stv_adaptors = []
for _ch, _adaptors in adaptor_dict.items():
    for _adaptor in _adaptors:
        if 'Stv' == _adaptor.id[:3]:
            stv_adaptors.append(_adaptor)
stv_adaptors = [_r for _r in sorted(stv_adaptors, key=lambda v:int(v.id.split('Stv_')[1].split('_')[0]))]

In [30]:
ndb_adaptors = []
for _ch, _adaptors in adaptor_dict.items():
    for _adaptor in _adaptors:
        if 'NDB' == _adaptor.id[:3]:
            ndb_adaptors.append(_adaptor)
ndb_adaptors = [_r for _r in sorted(ndb_adaptors, key=lambda v:int(v.id.split('NDB_')[1].split('_')[0]))]

In [31]:
ndb_adaptors

[SeqRecord(seq=Seq('GTTGACGGACTAATCCTGACTTTGCACTGCCGTCCTTGACTTTGCACTGCCGTCCTTGAC', SingleLetterAlphabet()), id='NDB_1_2xStv_82', name='<unknown name>', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq=Seq('GTTCAGACGCCGTTAAGCGCGATCCGATTGGAACCGTCCCGATCCGATTGGAACCGTCCC', SingleLetterAlphabet()), id='NDB_2_2xStv_1', name='<unknown name>', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq=Seq('GTGTGAGGCGCTAGAGCATATGCGAACTGTCCGGCTTTCATGCGAACTGTCCGGCTTTCA', SingleLetterAlphabet()), id='NDB_3_2xStv_79', name='<unknown name>', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq=Seq('GGCACGCCTAATCTGGACACTTTGCACTGCCGTCCTTGACTTTGCACTGCCGTCCTTGAC', SingleLetterAlphabet()), id='NDB_4_2xStv_82', name='<unknown name>', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq=Seq('TACGAGCGCTTGGATCCCGTGATCCGATTGGAACCGTCCCGATCCGATTGGAACCGTCCC', SingleLetterAlphabet()), id='NDB_5_2xStv_1', name='<unknown name>', description='<unknown description>', dbxre

# save

In [32]:
adaptor_folder

'\\\\10.245.74.212\\Chromatin_NAS_2\\Libraries\\Adaptors'

## save fasta

In [34]:
with open(os.path.join(adaptor_folder, 'Stv_adaptors.fasta'), 'w') as _output_handle:
    SeqIO.write(stv_adaptors, _output_handle, "fasta")

In [35]:
with open(os.path.join(adaptor_folder, 'NDB_adaptors.fasta'), 'w') as _output_handle:
    SeqIO.write(ndb_adaptors, _output_handle, "fasta")

## save csv

In [39]:
# generate csv file to order in IDT
import csv

with open(os.path.join(adaptor_folder, 'Stv_adaptor.csv'), 'w') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',', lineterminator='\n',
                       quotechar='|', quoting=csv.QUOTE_MINIMAL)
    # write header
    _header = ['Name', 'Sequence', 'Scale', 'Purification']
    csvwriter.writerow(_header)
    # write sequence
    for _adaptor in stv_adaptors:
        _info = [_adaptor.id, str(_adaptor.seq), '25nm', 'STD']
        csvwriter.writerow(_info)

In [38]:
# generate csv file to order in IDT
import csv

with open(os.path.join(adaptor_folder, 'NDB_adaptor.csv'), 'w') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',', lineterminator='\n',
                       quotechar='|', quoting=csv.QUOTE_MINIMAL)
    # write header
    _header = ['Name', 'Sequence', 'Scale', 'Purification']
    csvwriter.writerow(_header)
    # write sequence
    for _adaptor in ndb_adaptors:
        _info = [_adaptor.id, str(_adaptor.seq), '25nm', 'STD']
        csvwriter.writerow(_info)